### 다층 퍼셉트론(MultiLayer Perceptron, MLP)
* 구조
    * 다층 퍼셉트론과 단층 퍼셉트론의 차이는 다층 퍼셉트론은 입력층과 출력층 중간에 층을 더 추가하였다는데 이층을 은닉층(Hidden layer)이라고 한다.
    * 다층 퍼셉트론은 중간에 은닉층이 존재한다는 점이 단층 퍼셉트론과 다르다. 다층 퍼셉트론은 줄여서 MLP라고도 부른다
    * XOR 예제에서는 은닉층 1개만으로 문제를 해결할 수 있었지만, 다층 퍼셉트론은 본래 은닉층이 1개 이상인 퍼셉트론을 말한다.
    * XOR 문제보다 더욱 복잡한 문제를 해결하기 위해서 다층 퍼셉트론은 중간에 수많은 은닉층을 더 추가할 수 있다.
    * 은닉층이 2개 이상인 신경망을 심층 신경망(Deep Neural Network, DNN)이라고 한다.